In [ ]:
import numpy as np
from parse_lsp_data import *
import tensorflow as tf

In [ ]:
# from keras.preprocessing import image
from keras import applications
# from keras.applications.vgg16 import VGG16
# from keras.applications.vgg16 import preprocess_input
from keras.models import Model
# from keras.utils import np_utils
# from keras.models import Sequential
from keras.callbacks import CSVLogger
# from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout,Reshape
from keras.optimizers import Adam
import keras.backend as K
from keras.layers import Dense, Dropout, Conv2D, Flatten,Activation, Average
from image_noise_generation import *

In [ ]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [ ]:
# get the data set
train_list, train_label, val_list, val_label, test_list, test_label = getLSPDataset(0.9, 0.05)

In [ ]:
# print some information
train_X = train_list	# (N, 227, 227, 3)
train_Y = train_label	# (N, 2, 14)
# train_X,train_Y = data_augmentation(train_X,train_Y)
print("train X: ", train_X.shape, "val X: ", val_list.shape, "test X: ", test_list.shape)
print("train Y: ", train_Y.shape, "val Y: ", val_label.shape, "test Y: ", test_label.shape)

In [ ]:
# model_input = Input(shape=(227,227,3))

In [ ]:
# build keras model
def build_model():
    base_model = applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(227,227, 3))
    last = base_model.output
    x = Flatten()(last)
#     x = Dropout(0.7)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.7)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.7)(x)
    x = Dense(28, activation='relu')(x)
    preds = Reshape((2,14))(x)
    model = Model(base_model.input, preds)
    return model

In [ ]:
model = build_model()

In [ ]:
#model.summary()

In [ ]:
# custom loss function (L2 distance)
def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_pred - y_true)))/64

In [ ]:
thresh = 0.5
def accuracy(y_true, y_pred):
    
    diff = K.square(y_true - y_pred)
#     loss = K.sum(diff)
    dist = K.sqrt(K.sum(diff, axis = 1))	# (N, 2, 14) -> (N, 14)
#     accuracy = tf.reduce_sum(tf.to_int32(tf.greater(1.0, dist))) / 64/14
    torsor_xy = y_true[:,:,9] - y_true[:,:,2] # (N,2)
    torsor_dist = K.sqrt(K.sum(K.square(torsor_xy),axis=1,keepdims=True)) #(N,1)
    torsor_frac = thresh*torsor_dist
    
    accuracy = K.sum(K.cast(K.greater(torsor_frac,dist),dtype='float32'))/64/14
    
    return accuracy

In [ ]:
# compile the model
model.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss, metrics=[accuracy])

In [ ]:
csv_logger = CSVLogger('resnet50_t05_lr00005_dp09_v2.csv', append=True, separator=';')

In [ ]:
with tf.device('/device:GPU:0'):
    # fit the model
    history = model.fit(train_X, train_Y, epochs=100,batch_size=64, validation_data = (val_list,val_label),callbacks=[csv_logger])

In [ ]:
# evaluation
with tf.device('/device:GPU:0'):
    loss,acc = model.evaluate(test_list, test_label,batch_size=64)

In [ ]:
print(loss)
print(acc)

In [ ]:
with tf.device('/device:GPU:0'):
    pred = model.predict(test_list)with tf.device('/device:GPU:0'):

In [ ]:
print(pred[0])
print(test_label[0])

In [ ]:
dist = K.sqrt(K.sum(K.square(pred[0] - test_label[0]), axis=0))

In [ ]:
threshold = np.array(1.0)

In [ ]:
temp = tf.reduce_sum(tf.to_int32(tf.greater(threshold, dist)))

In [ ]:
print(temp.shape)

In [ ]:
model.save('resnet50_t05_lr00005_dp09_v2.h5')

In [ ]:
# load model
def load_trained_model(weights_path):
    model = build_model()
    model.load_weights(weights_path)
    return model

In [ ]:
model = load_trained_model('resnet50_t05_lr00005_v2.h5')

In [ ]:
model.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss, metrics=[accuracy])

In [ ]:
pred_test = model.predict(test_list)

In [ ]:
# pred2 = train_model.predict(train_list)
pred_train = model.predict(train_list)

In [ ]:
pred_val = model.predict(val_list)

In [ ]:
# usage:
# drawLines(val_list[5].copy(),val_label[5].copy(),pred3[5].copy())

In [ ]:
from drawLines_v2 import drawLines
# usage:
index = 5
drawLines(test_list[index].copy(),test_label[index].copy(),pred_test[index].copy())

In [ ]:
from drawLines_v2 import drawLines
index = 71
drawLines(val_list[index].copy(),val_label[index].copy(),pred_val[index].copy())

In [ ]:
# usage:
index2 = 5
drawLines(train_list[index2].copy(),train_label[index2].copy(),pred_train[index2].copy())

In [ ]:
# compute ankle validation acc
thresh = 0.0

diff = np.square(val_label - pred_val)
#     loss = K.sum(diff)
dist = np.sqrt(np.sum(diff, axis = 1))[:,[7,10]]	# (N, 2, 14) -> (N, 14)
#     accuracy = tf.reduce_sum(tf.to_int32(tf.greater(1.0, dist))) / 64/14
torsor_xy = val_label[:,:,9] - val_label[:,:,2] # (N,2)
torsor_dist = np.sqrt(np.sum(np.square(torsor_xy),axis=1,keepdims=True)) #(N,1)
torsor_frac = thresh*torsor_dist

test_accuracy = np.sum(np.greater(torsor_frac,dist))/200/2
print(test_accuracy)

In [ ]:
# compute knee validation acc

In [ ]:
# compute wrists validation acc

In [ ]:
# compute elbows validation acc

In [ ]:
# # load model
# def load_trained_model(weights_path):
#     model = build_model()
#     model.load_weights(weights_path)
#     return model

In [ ]:
# model = load_trained_model('resnet50_t05_lr00005.h5')
# model.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss, metrics=[accuracy])

In [ ]:
# # cascade
# with tf.device('/device:GPU:0'):
#     pred = model.predict(train_list)

In [ ]:
# with tf.device('/device:GPU:0'):
#     pred_val = model.predict(val_list)

In [ ]:
from crop_image_keras_v2 import images_crop

In [ ]:
crop_image_matrix,actual_reshape_size_matrix = images_crop(train_list.copy(),pred_train.copy(),train_label.copy())

In [ ]:
crop_image_matrix_val,actual_reshape_size_matrix_val = images_crop(val_list.copy(),pred_val.copy(),val_label.copy())

In [ ]:
# with tf.device('/device:GPU:0'):
#     crop_image_matrix = crop_image_matrix.transpose(0,2,3,4,1).reshape(1600,227,227,-1)

In [ ]:
# with tf.device('/device:GPU:0'):
#     crop_image_matrix_val = crop_image_matrix_val.transpose(0,2,3,4,1).reshape(200,227,227,-1)

In [ ]:
# put reshape information and previous stage's prediction into y_true, this is an implementation trick
# in this case we have reshape information and previous prediction along each batch
stage2_Y = np.concatenate((train_Y,actual_reshape_size_matrix,pred_train),axis=1)
stage2_Y_val = np.concatenate((val_label,actual_reshape_size_matrix_val,pred_val),axis=1)

In [ ]:
# build keras model
# def stage2():
#     base_model = applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(227,227, 3,14))
#     last = base_model.output
#     x = Flatten()(last)
# #     x = Dropout(0.7)(x)
#     x = Dense(4096, activation='relu')(x)
#     x = Dropout(0.7)(x)
#     x = Dense(4096, activation='relu')(x)
#     x = Dropout(0.7)(x)
#     x = Dense(28, activation='relu')(x)
#     preds = Reshape((2,14))(x)
#     model = Model(base_model.input, preds)
#     return model

In [ ]:
# stage2_model = stage2()

In [ ]:
# build keras model
def build_cascade_model():
    base_model = applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(227,227, 3))
    last = base_model.output
    x = Flatten()(last)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.7)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.7)(x)
    x = Dense(84, activation='relu')(x)
    preds = Reshape((6,14))(x) # an implementation trick here. Only 2*14 is used for displacement prediction. The result is to match the shape of y_true as stated above
    model = Model(base_model.input, preds)
    return model

In [ ]:
# custom loss function (L2 distance)
def euclidean_distance_loss_stageS(y_true, y_pred):
    
    y_true_labels = y_true[:,:2,:]  # (N,2,14)
    reshape_matrix = y_true[:,2:4,:] # (N,2,14)
    y_pred_prev = y_true[:,4:,:] # (N,2,14)
    y_pred_resize = y_pred_prev + y_pred[:,:2,:]*reshape_matrix/227
    return K.sqrt(K.sum(K.square(y_pred_resize - y_true_labels)))/64

In [ ]:
thresh = 0.5
def accuracy(y_true, y_pred):
    y_true_labels = y_true[:,:2,:]  # (N,2,14)
    reshape_matrix = y_true[:,2:4,:] # (N,2,14)
    y_pred_prev = y_true[:,4:,:] # (N,2,14)
    
    y_pred_resize = y_pred_prev + y_pred[:,:2,:]*reshape_matrix/227
    
    diff = K.square(y_true_labels - y_pred_resize)
#     loss = K.sum(diff)
    dist = K.sqrt(K.sum(diff, axis = 1))	# (N, 2, 14) -> (N, 14)
#     accuracy = tf.reduce_sum(tf.to_int32(tf.greater(1.0, dist))) / 64/14
    torsor_xy = y_true_labels[:,:,9] - y_true_labels[:,:,2] # (N,2)
    torsor_dist = K.sqrt(K.sum(K.square(torsor_xy),axis=1,keepdims=True)) #(N,1)
    torsor_frac = thresh*torsor_dist
    
    accuracy = K.sum(K.cast(K.greater(torsor_frac,dist),dtype='float32'))/64/14
    
    return accuracy

In [ ]:
train_pred_matrix = np.zeros((1600,2,14))
val_pred_matrix = np.zeros((200,2,14))

In [ ]:
# first joint
temp_images0 = crop_image_matrix[:,0]
temp_images_val = crop_image_matrix_val[:,0]
cascade_model0 = build_cascade_model()

In [ ]:
cascade_model0.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss_stageS, metrics=[accuracy])

In [ ]:
csv_logger = CSVLogger('stage2_0_v4.csv', append=True, separator=';')

In [ ]:
with tf.device('/device:GPU:0'):
    # fit the model
    cascade_model0.fit(temp_images0, stage2_Y, epochs=20,batch_size=64,validation_data = (temp_images_val,stage2_Y_val),callbacks=[csv_logger])

In [ ]:
cascade_model0.save('stage2_0_v4.h5')

In [ ]:
# store training prediction
with tf.device('/device:GPU:0'):
    temp_pred2_train0 = cascade_model0.predict(temp_images0) # (N,6,14), but only (N,:2,14) are useful information. 
train_pred_matrix[:,:,0] = temp_pred2_train0[:,:2,0] + pred_train[:,:,0] # displacement + previous prediction
with tf.device('/device:GPU:0'):
    temp_pred2_val0 = cascade_model0.predict(temp_images_val) # (N_prime,6,14)
val_pred_matrix[:,:,0] = temp_pred2_val0[:,:2,0] + pred_val[:,:,0]

In [ ]:
# second joint
index = 1
temp_images1 = crop_image_matrix[:,index]
temp_images_val1 = crop_image_matrix_val[:,index]
cascade_model1 = build_cascade_model()
cascade_model1.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss_stageS, metrics=[accuracy])
csv_logger = CSVLogger('stage2_1_v4.csv', append=True, separator=';')

In [ ]:
with tf.device('/device:GPU:0'):
    # fit the model
    cascade_model1.fit(temp_images1, stage2_Y, epochs=20,batch_size=64,validation_data = (temp_images_val1,stage2_Y_val),callbacks=[csv_logger])

In [ ]:
cascade_model1.save('stage2_1_v4.h5')

In [ ]:
# store training prediction
with tf.device('/device:GPU:0'):
    temp_pred2_train1 = cascade_model1.predict(temp_images1) # (N,6,14), but only (N,:2,14) are useful information. 
train_pred_matrix[:,:,index] = temp_pred2_train1[:,:2,index] + pred_train[:,:,index]
with tf.device('/device:GPU:0'):
    temp_pred2_val1 = cascade_model1.predict(temp_images_val1) # (N_prime,6,14)
val_pred_matrix[:,:,index] = temp_pred2_val1[:,:2,index] + pred_val[:,:,index]

In [ ]:
# third joint
index = 2
temp_images2 = crop_image_matrix[:,index]
temp_images_val2 = crop_image_matrix_val[:,index]
cascade_model2 = build_cascade_model()
cascade_model2.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss_stageS, metrics=[accuracy])
csv_logger = CSVLogger('stage2_2_v4.csv', append=True, separator=';')

In [ ]:
with tf.device('/device:GPU:0'):
    # fit the model
    cascade_model2.fit(temp_images2, stage2_Y, epochs=20,batch_size=64,validation_data = (temp_images_val2,stage2_Y_val),callbacks=[csv_logger])

In [ ]:
cascade_model2.save('stage2_2_v4.h5')

In [ ]:
# store training prediction
with tf.device('/device:GPU:0'):
    temp_pred2_train2 = cascade_model2.predict(temp_images2) # (N,6,14), but only (N,:2,14) are useful information. 
train_pred_matrix[:,:,index] = temp_pred2_train2[:,:2,index] + pred_train[:,:,index]
with tf.device('/device:GPU:0'):
    temp_pred2_val2 = cascade_model2.predict(temp_images_val2) # (N_prime,6,14)
val_pred_matrix[:,:,index] = temp_pred2_val2[:,:2,index] + pred_val[:,:,index]

In [ ]:
# forth joint
index = 3
temp_images3 = crop_image_matrix[:,index]
temp_images_val3 = crop_image_matrix_val[:,index]
cascade_model3 = build_cascade_model()
cascade_model3.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss_stageS, metrics=[accuracy])
csv_logger = CSVLogger('stage2_3_v4.csv', append=True, separator=';')

In [ ]:
with tf.device('/device:GPU:0'):
    # fit the model
    cascade_model3.fit(temp_images3, stage2_Y, epochs=20,batch_size=64,validation_data = (temp_images_val3,stage2_Y_val),callbacks=[csv_logger])

In [ ]:
cascade_model3.save('stage2_3_v4.h5')
# store training prediction
with tf.device('/device:GPU:0'):
    temp_pred2_train3 = cascade_model3.predict(temp_images3) # (N,6,14), but only (N,:2,14) are useful information. 
train_pred_matrix[:,:,index] = temp_pred2_train3[:,:2,index] + pred_train[:,:,index]
with tf.device('/device:GPU:0'):
    temp_pred2_val3 = cascade_model3.predict(temp_images_val3) # (N_prime,6,14)
val_pred_matrix[:,:,index] = temp_pred2_val3[:,:2,index] + pred_val[:,:,index]

In [ ]:
# fifth joint
index = 4
temp_images4 = crop_image_matrix[:,index]
temp_images_val4 = crop_image_matrix_val[:,index]
cascade_model4 = build_cascade_model()
cascade_model4.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss_stageS, metrics=[accuracy])
csv_logger = CSVLogger('stage2_4_v4.csv', append=True, separator=';')

In [ ]:
with tf.device('/device:GPU:0'):
    # fit the model
    cascade_model4.fit(temp_images4, stage2_Y, epochs=20,batch_size=64,validation_data = (temp_images_val4,stage2_Y_val),callbacks=[csv_logger])

In [ ]:
cascade_model4.save('stage2_4_v4.h5')
# store training prediction
with tf.device('/device:GPU:0'):
    temp_pred2_train4 = cascade_model4.predict(temp_images4) # (N,6,14), but only (N,:2,14) are useful information. 
train_pred_matrix[:,:,index] = temp_pred2_train4[:,:2,index] + pred_train[:,:,index]
with tf.device('/device:GPU:0'):
    temp_pred2_val4 = cascade_model4.predict(temp_images_val4) # (N_prime,6,14)
val_pred_matrix[:,:,index] = temp_pred2_val4[:,:2,index] + pred_val[:,:,index]

In [ ]:
#sixth joint
index = 5
temp_images5 = crop_image_matrix[:,index]
temp_images_val5 = crop_image_matrix_val[:,index]
cascade_model5 = build_cascade_model()
cascade_model5.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss_stageS, metrics=[accuracy])
csv_logger = CSVLogger('stage2_5_v4.csv', append=True, separator=';')

In [ ]:
with tf.device('/device:GPU:0'):
    # fit the model
    cascade_model5.fit(temp_images5, stage2_Y, epochs=20,batch_size=64,validation_data = (temp_images_val5,stage2_Y_val),callbacks=[csv_logger])

In [ ]:
cascade_model5.save('stage2_5_v4.h5')
# store training prediction
with tf.device('/device:GPU:0'):
    temp_pred2_train5 = cascade_model5.predict(temp_images5) # (N,6,14), but only (N,:2,14) are useful information. 
train_pred_matrix[:,:,index] = temp_pred2_train5[:,:2,index] + pred_train[:,:,index]
with tf.device('/device:GPU:0'):
    temp_pred2_val5 = cascade_model5.predict(temp_images_val5) # (N_prime,6,14)
val_pred_matrix[:,:,index] = temp_pred2_val5[:,:2,index] + pred_val[:,:,index]

In [ ]:
# seventh joint
index = 6
temp_images6 = crop_image_matrix[:,index]
temp_images_val6 = crop_image_matrix_val[:,index]
cascade_model6 = build_cascade_model()
cascade_model6.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss_stageS, metrics=[accuracy])
csv_logger = CSVLogger('stage2_6_v4.csv', append=True, separator=';')

In [ ]:
with tf.device('/device:GPU:0'):
    # fit the model
    cascade_model6.fit(temp_images6, stage2_Y, epochs=20,batch_size=64,validation_data = (temp_images_val6,stage2_Y_val),callbacks=[csv_logger])

In [ ]:
cascade_model6.save('stage2_6_v4.h5')
# store training prediction
with tf.device('/device:GPU:0'):
    temp_pred2_train6 = cascade_model6.predict(temp_images6) # (N,6,14), but only (N,:2,14) are useful information. 
train_pred_matrix[:,:,index] = temp_pred2_train6[:,:2,index] + pred_train[:,:,index]
with tf.device('/device:GPU:0'):
    temp_pred2_val6 = cascade_model6.predict(temp_images_val6) # (N_prime,6,14)
val_pred_matrix[:,:,index] = temp_pred2_val6[:,:2,index] + pred_val[:,:,index]

In [ ]:
# eighth joint
index = 7
temp_images7 = crop_image_matrix[:,index]
temp_images_val7 = crop_image_matrix_val[:,index]
cascade_model7 = build_cascade_model()
cascade_model7.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss_stageS, metrics=[accuracy])
csv_logger = CSVLogger('stage2_7_v4.csv', append=True, separator=';')

In [ ]:
with tf.device('/device:GPU:0'):
    # fit the model
    cascade_model7.fit(temp_images7, stage2_Y, epochs=20,batch_size=64,validation_data = (temp_images_val7,stage2_Y_val),callbacks=[csv_logger])

In [ ]:
cascade_model7.save('stage2_7_v4.h5')
# store training prediction
with tf.device('/device:GPU:0'):
    temp_pred2_train7 = cascade_model7.predict(temp_images7) # (N,6,14), but only (N,:2,14) are useful information. 
train_pred_matrix[:,:,index] = temp_pred2_train7[:,:2,index] + pred_train[:,:,index]
with tf.device('/device:GPU:0'):
    temp_pred2_val7 = cascade_model7.predict(temp_images_val7) # (N_prime,6,14)
val_pred_matrix[:,:,index] = temp_pred2_val7[:,:2,index] + pred_val[:,:,index]

In [ ]:
# ninth joint
index = 8
temp_images8 = crop_image_matrix[:,index]
temp_images_val8 = crop_image_matrix_val[:,index]
cascade_model8 = build_cascade_model()
cascade_model8.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss_stageS, metrics=[accuracy])
csv_logger = CSVLogger('stage2_8_v4.csv', append=True, separator=';')

In [ ]:
with tf.device('/device:GPU:0'):
    # fit the model
    cascade_model8.fit(temp_images8, stage2_Y, epochs=20,batch_size=64,validation_data = (temp_images_val8,stage2_Y_val),callbacks=[csv_logger])

In [ ]:
cascade_model8.save('stage2_8_v4.h5')
# store training prediction
with tf.device('/device:GPU:0'):
    temp_pred2_train8 = cascade_model8.predict(temp_images8) # (N,6,14), but only (N,:2,14) are useful information. 
train_pred_matrix[:,:,index] = temp_pred2_train8[:,:2,index] + pred_train[:,:,index]
with tf.device('/device:GPU:0'):
    temp_pred2_val8 = cascade_model8.predict(temp_images_val8) # (N_prime,6,14)
val_pred_matrix[:,:,index] = temp_pred2_val8[:,:2,index] + pred_val[:,:,index]

In [ ]:
# tenth joint
index = 9
temp_images9 = crop_image_matrix[:,index]
temp_images_val9 = crop_image_matrix_val[:,index]
cascade_model9 = build_cascade_model()
cascade_model9.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss_stageS, metrics=[accuracy])
csv_logger = CSVLogger('stage2_9_v4.csv', append=True, separator=';')

In [ ]:
with tf.device('/device:GPU:0'):
    # fit the model
    cascade_model9.fit(temp_images9, stage2_Y, epochs=20,batch_size=64,validation_data = (temp_images_val9,stage2_Y_val),callbacks=[csv_logger])

In [ ]:
cascade_model9.save('stage2_9_v4.h5')
# store training prediction
with tf.device('/device:GPU:0'):
    temp_pred2_train9 = cascade_model9.predict(temp_images9) # (N,6,14), but only (N,:2,14) are useful information. 
train_pred_matrix[:,:,index] = temp_pred2_train9[:,:2,index] + pred_train[:,:,index]
with tf.device('/device:GPU:0'):
    temp_pred2_val9 = cascade_model9.predict(temp_images_val9) # (N_prime,6,14)
val_pred_matrix[:,:,index] = temp_pred2_val9[:,:2,index] + pred_val[:,:,index]

In [ ]:
# eleventh joint
index = 10
temp_images10 = crop_image_matrix[:,index]
temp_images_val10 = crop_image_matrix_val[:,index]
cascade_model10 = build_cascade_model()
cascade_model10.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss_stageS, metrics=[accuracy])
csv_logger = CSVLogger('stage2_10_v4.csv', append=True, separator=';')

In [ ]:
with tf.device('/device:GPU:0'):
    # fit the model
    cascade_model10.fit(temp_images10, stage2_Y, epochs=20,batch_size=64,validation_data = (temp_images_val10,stage2_Y_val),callbacks=[csv_logger])

In [ ]:
cascade_model10.save('stage2_10_v4.h5')
# store training prediction
with tf.device('/device:GPU:0'):
    temp_pred2_train10 = cascade_model10.predict(temp_images10) # (N,6,14), but only (N,:2,14) are useful information. 
train_pred_matrix[:,:,index] = temp_pred2_train10[:,:2,index] + pred_train[:,:,index]
with tf.device('/device:GPU:0'):
    temp_pred2_val10 = cascade_model10.predict(temp_images_val10) # (N_prime,6,14)
val_pred_matrix[:,:,index] = temp_pred2_val10[:,:2,index] + pred_val[:,:,index]

In [ ]:
# 12th joint
index = 11
temp_images11 = crop_image_matrix[:,index]
temp_images_val11 = crop_image_matrix_val[:,index]
cascade_model11 = build_cascade_model()
cascade_model11.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss_stageS, metrics=[accuracy])
csv_logger = CSVLogger('stage2_11_v4.csv', append=True, separator=';')

In [ ]:
with tf.device('/device:GPU:0'):
    # fit the model
    cascade_model11.fit(temp_images11, stage2_Y, epochs=20,batch_size=64,validation_data = (temp_images_val11,stage2_Y_val),callbacks=[csv_logger])

In [ ]:
cascade_model11.save('stage2_11_v4.h5')
# store training prediction
with tf.device('/device:GPU:0'):
    temp_pred2_train11 = cascade_model11.predict(temp_images11) # (N,6,14), but only (N,:2,14) are useful information. 
train_pred_matrix[:,:,index] = temp_pred2_train11[:,:2,index] + pred_train[:,:,index]
with tf.device('/device:GPU:0'):
    temp_pred2_val11 = cascade_model11.predict(temp_images_val11) # (N_prime,6,14)
val_pred_matrix[:,:,index] = temp_pred2_val11[:,:2,index] + pred_val[:,:,index]

In [ ]:
# 13th joint
index = 12
temp_images12 = crop_image_matrix[:,index]
temp_images_val12 = crop_image_matrix_val[:,index]
cascade_model12 = build_cascade_model()
cascade_model12.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss_stageS, metrics=[accuracy])
csv_logger = CSVLogger('stage2_12_v4.csv', append=True, separator=';')

In [ ]:
with tf.device('/device:GPU:0'):
    # fit the model
    cascade_model12.fit(temp_images12, stage2_Y, epochs=20,batch_size=64,validation_data = (temp_images_val12,stage2_Y_val),callbacks=[csv_logger])

In [ ]:
cascade_model12.save('stage2_12_v4.h5')
# store training prediction
with tf.device('/device:GPU:0'):
    temp_pred2_train12 = cascade_model12.predict(temp_images12) # (N,6,14), but only (N,:2,14) are useful information. 
train_pred_matrix[:,:,index] = temp_pred2_train12[:,:2,index] + pred_train[:,:,index]
with tf.device('/device:GPU:0'):
    temp_pred2_val12 = cascade_model12.predict(temp_images_val12) # (N_prime,6,14)
val_pred_matrix[:,:,index] = temp_pred2_val12[:,:2,index] + pred_val[:,:,index]

In [ ]:
# 14th joint
index = 13
temp_images13 = crop_image_matrix[:,index]
temp_images_val13 = crop_image_matrix_val[:,index]
cascade_model13 = build_cascade_model()
cascade_model13.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss_stageS, metrics=[accuracy])
csv_logger = CSVLogger('stage2_13_v4.csv', append=True, separator=';')

In [ ]:
with tf.device('/device:GPU:0'):
    # fit the model
    cascade_model13.fit(temp_images13, stage2_Y, epochs=20,batch_size=64,validation_data = (temp_images_val13,stage2_Y_val),callbacks=[csv_logger])

In [ ]:
cascade_model13.save('stage2_13_v4.h5')
# store training prediction
with tf.device('/device:GPU:0'):
    temp_pred2_train13 = cascade_model13.predict(temp_images13) # (N,6,14), but only (N,:2,14) are useful information. 
train_pred_matrix[:,:,index] = temp_pred2_train13[:,:2,index] + pred_train[:,:,index]
with tf.device('/device:GPU:0'):
    temp_pred2_val13 = cascade_model12.predict(temp_images_val13) # (N_prime,6,14)
val_pred_matrix[:,:,index] = temp_pred2_val13[:,:2,index] + pred_val[:,:,index]

In [ ]:
# compute final train and val loss
final_loss_train = np.sqrt(np.sum(np.square(train_Y - train_pred_matrix)))/1600
print(final_loss_train)
final_loss_val = np.sqrt(np.sum(np.square(val_label - val_pred_matrix)))/200
print(final_loss_val)

In [ ]:
# compute final train acc
thresh = 0.5
   
diff = np.square(train_Y - train_pred_matrix)
#     loss = K.sum(diff)
dist = np.sqrt(np.sum(diff, axis = 1))	# (N, 2, 14) -> (N, 14)
#     accuracy = tf.reduce_sum(tf.to_int32(tf.greater(1.0, dist))) / 64/14
torsor_xy = train_Y[:,:,9] - train_Y[:,:,2] # (N,2)
torsor_dist = np.sqrt(np.sum(np.square(torsor_xy),axis=1,keepdims=True)) #(N,1)
torsor_frac = thresh*torsor_dist

train_accuracy = np.sum(np.greater(torsor_frac,dist))/1600/14
print(train_accuracy)

In [ ]:
# compute final val acc
thresh = 0.5
   
diff = np.square(val_label - val_pred_matrix)
#     loss = K.sum(diff)
dist = np.sqrt(np.sum(diff, axis = 1))	# (N, 2, 14) -> (N, 14)
#     accuracy = tf.reduce_sum(tf.to_int32(tf.greater(1.0, dist))) / 64/14
torsor_xy = val_label[:,:,9] - val_label[:,:,2] # (N,2)
torsor_dist = np.sqrt(np.sum(np.square(torsor_xy),axis=1,keepdims=True)) #(N,1)
torsor_frac = thresh*torsor_dist

val_accuracy = np.sum(np.greater(torsor_frac,dist))/200/14
print(val_accuracy)

In [ ]:
crop_image_matrix_test,actual_reshape_size_matrix_test = images_crop(test_list.copy(),pred_test.copy(),test_label.copy())

In [ ]:
test_images0 = crop_image_matrix_test[:,0]
test_images1 = crop_image_matrix_test[:,1]
test_images2 = crop_image_matrix_test[:,2]
test_images3 = crop_image_matrix_test[:,3]
test_images4 = crop_image_matrix_test[:,4]
test_images5 = crop_image_matrix_test[:,5]
test_images6 = crop_image_matrix_test[:,6]
test_images7 = crop_image_matrix_test[:,7]
test_images8 = crop_image_matrix_test[:,8]
test_images9 = crop_image_matrix_test[:,9]
test_images10 = crop_image_matrix_test[:,10]
test_images11 = crop_image_matrix_test[:,11]
test_images12 = crop_image_matrix_test[:,12]
test_images13 = crop_image_matrix_test[:,13]

test_pred_matrix = np.zeros((200,2,14))

In [ ]:
with tf.device('/device:GPU:0'):
    temp_pred2_test0 = cascade_model0.predict(test_images0)
    temp_pred2_test1 = cascade_model1.predict(test_images1)
    temp_pred2_test2 = cascade_model2.predict(test_images2)
    temp_pred2_test3 = cascade_model3.predict(test_images3)
    temp_pred2_test4 = cascade_model4.predict(test_images4)
    temp_pred2_test5 = cascade_model5.predict(test_images5)
    temp_pred2_test6 = cascade_model6.predict(test_images6)
    temp_pred2_test7 = cascade_model7.predict(test_images7)
    temp_pred2_test8 = cascade_model8.predict(test_images8)
    temp_pred2_test9 = cascade_model9.predict(test_images9)
    temp_pred2_test10 = cascade_model10.predict(test_images10)
    temp_pred2_test11 = cascade_model11.predict(test_images11)
    temp_pred2_test12 = cascade_model12.predict(test_images12)
    temp_pred2_test13 = cascade_model13.predict(test_images13)

test_pred_matrix[:,:,0] = temp_pred2_test0[:,:2,0] + pred_test[:,:,0]
test_pred_matrix[:,:,1] = temp_pred2_test1[:,:2,1] + pred_test[:,:,1]
test_pred_matrix[:,:,2] = temp_pred2_test2[:,:2,2] + pred_test[:,:,2]
test_pred_matrix[:,:,3] = temp_pred2_test3[:,:2,3] + pred_test[:,:,3]
test_pred_matrix[:,:,4] = temp_pred2_test4[:,:2,4] + pred_test[:,:,4]
test_pred_matrix[:,:,5] = temp_pred2_test5[:,:2,5] + pred_test[:,:,5]
test_pred_matrix[:,:,6] = temp_pred2_test6[:,:2,6] + pred_test[:,:,6]
test_pred_matrix[:,:,7] = temp_pred2_test7[:,:2,7] + pred_test[:,:,7]
test_pred_matrix[:,:,8] = temp_pred2_test8[:,:2,8] + pred_test[:,:,8]
test_pred_matrix[:,:,9] = temp_pred2_test9[:,:2,9] + pred_test[:,:,9]
test_pred_matrix[:,:,10] = temp_pred2_test10[:,:2,10] + pred_test[:,:,10]
test_pred_matrix[:,:,11] = temp_pred2_test11[:,:2,11] + pred_test[:,:,11]
test_pred_matrix[:,:,12] = temp_pred2_test12[:,:2,12] + pred_test[:,:,12]
test_pred_matrix[:,:,13] = temp_pred2_test13[:,:2,13] + pred_test[:,:,13]

In [ ]:
# compute final test loss
final_loss_test = np.sqrt(np.sum(np.square(test_label - test_pred_matrix)))/200
print(final_loss_test)

In [ ]:
# compute final test acc
thresh = 0.5
   
diff = np.square(test_label - test_pred_matrix)
#     loss = K.sum(diff)
dist = np.sqrt(np.sum(diff, axis = 1))	# (N, 2, 14) -> (N, 14)
#     accuracy = tf.reduce_sum(tf.to_int32(tf.greater(1.0, dist))) / 64/14
torsor_xy = test_label[:,:,9] - test_label[:,:,2] # (N,2)
torsor_dist = np.sqrt(np.sum(np.square(torsor_xy),axis=1,keepdims=True)) #(N,1)
torsor_frac = thresh*torsor_dist

test_accuracy = np.sum(np.greater(torsor_frac,dist))/200/14
print(test_accuracy)

In [ ]:
# print(crop_image_matrix.shape)
# print(actual_reshape_size_matrix.shape)
# # print(scaled_preds_label.shape)
# print(scaled_trues_label.shape)
# print(train_list.shape)
# print(pred_train.shape)
# print(train_label.shape)

# print(crop_image_matrix_val.shape)
# print(actual_reshape_size_matrix_val.shape)

# print(actual_reshape_size_matrix[0])
# print(actual_reshape_size_matrix_val[0])
# print(scaled_preds_label[0,0])
# print(pred_train[0])
# print(scaled_trues_label[0,0])
# print(train_label[0])

In [ ]:
# build keras model
# def build_cascade_model():
#     base_model = applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(227,227, 3))
#     last = base_model.output
#     x = Flatten()(last)
#     x = Dense(4096, activation='relu')(x)
#     x = Dropout(0.7)(x)
#     x = Dense(4096, activation='relu')(x)
#     x = Dropout(0.7)(x)
#     x = Dense(28, activation='relu')(x)
#     preds = Reshape((2,14))(x)
#     model = Model(base_model.input, preds)
#     return model

In [ ]:
# # custom loss function (L2 distance)
# def euclidean_distance_loss(y_true, y_pred):
#     return K.sqrt(K.sum(K.square(y_pred - y_true)))/64

In [ ]:
# train_pred_matrix = np.zeros((1600,2,14))
# val_pred_matrix = np.zeros((200,2,14))

In [ ]:
# put model separately and have a test
####################################################

In [ ]:
# temp_images0 = crop_image_matrix[:,0]
# temp_true_labels0 = scaled_trues_label[:,0,:]
# temp_images_val = crop_image_matrix_val[:,0]
# temp_true_labels0_val = scaled_trues_label_val[:,0,:]
# cascade_model0 = build_cascade_model()
# cascade_model0.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss, metrics=[])

In [ ]:
# csv_logger = CSVLogger('stage2_0_v2.csv', append=True, separator=';')

In [ ]:
# with tf.device('/device:GPU:0'):
    # fit the model
#     history = cascade_model0.fit(temp_images0, temp_true_labels0, epochs=50,batch_size=64,validation_data = (temp_images_val,temp_true_labels0_val),callbacks=[csv_logger])

In [ ]:
# cascade_model0.save('stage2_0_v2.h5')

In [ ]:
# store training prediction
# with tf.device('/device:GPU:0'):
# temp_pred2_train0 = cascade_model0.predict(temp_images0) # (1600,2,14)

In [ ]:
# scale back to original images to compute loss
# temp_pred2_train0_store = temp_pred2_train0*actual_reshape_size_matrix/227

In [ ]:
# train_pred_matrix[:,:,0] = temp_pred2_train0_store[:,:,0]

In [ ]:
# temp_val_images0 = crop_image_matrix_val[:,0]
# with tf.device('/device:GPU:0'):
# temp_pred2_val0 = cascade_model0.predict(temp_val_images0) # (200,2,1)

In [ ]:
# temp_pred2_val0_store = temp_pred2_val0*actual_reshape_size_matrix_val/227

In [ ]:
# val_pred_matrix[:,:,0] = temp_pred2_val0_store[:,:,0]

In [ ]:
# second joint
# index = 1
# temp_images1 = crop_image_matrix[:,index]
# temp_true_labels1 = scaled_trues_label[:,index,:]
# temp_images_val1 = crop_image_matrix_val[:,index]
# temp_true_labels1_val = scaled_trues_label_val[:,index,:]
# cascade_model1 = build_cascade_model()
# cascade_model1.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss, metrics=[])

In [ ]:
# csv_logger = CSVLogger('stage2_1_v2.csv', append=True, separator=';')

In [ ]:
# with tf.device('/device:GPU:0'):
    # fit the model
#     history = cascade_model1.fit(temp_images1, temp_true_labels1, epochs=50,batch_size=64,validation_data = (temp_images_val1,temp_true_labels1_val),callbacks=[csv_logger])

In [ ]:
# cascade_model1.save('stage2_1_v2.h5')

In [ ]:
# store training prediction
# with tf.device('/device:GPU:0'):
#     temp_pred2_train1 = cascade_model1.predict(temp_images1) # (1600,2,1)
# # scale back to original images to compute loss
# temp_pred2_train1_store = temp_pred2_train1*actual_reshape_size_matrix/227
# train_pred_matrix[:,:,index] = temp_pred2_train1_store[:,:,index]
# temp_val_images1 = crop_image_matrix_val[:,index]
# with tf.device('/device:GPU:0'):
#     temp_pred2_val1 = cascade_model1.predict(temp_val_images1) # (200,2,1)
# temp_pred2_val1_store = temp_pred2_val1*actual_reshape_size_matrix_val/227
# val_pred_matrix[:,:,index] = temp_pred2_val1_store[:,:,index]

In [ ]:
# third joint
# index = 2
# temp_images2 = crop_image_matrix[:,index]
# temp_true_labels2 = scaled_trues_label[:,index,:]
# temp_images_val2 = crop_image_matrix_val[:,index]
# temp_true_labels2_val = scaled_trues_label_val[:,index,:]
# cascade_model2 = build_cascade_model()
# cascade_model2.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss, metrics=[])

In [ ]:
# csv_logger = CSVLogger('stage2_2_v2.csv', append=True, separator=';')

In [ ]:
# cascade_model2.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss, metrics=[])

In [ ]:
# with tf.device('/device:GPU:0'):
    # fit the model
#     cascade_model2.fit(temp_images2, temp_true_labels2, epochs=50,batch_size=64,validation_data = (temp_images_val2,temp_true_labels2_val),callbacks=[csv_logger])

In [ ]:
# cascade_model2.save('stage2_2_v2.h5')

In [ ]:
# store training prediction
# with tf.device('/device:GPU:0'):
#     temp_pred2_train2 = cascade_model2.predict(temp_images2) # (1600,2,1)
# # scale back to original images to compute loss
# temp_pred2_train2_store = temp_pred2_train2*actual_reshape_size_matrix/227
# train_pred_matrix[:,:,index] = temp_pred2_train2_store[:,:,index]
# temp_val_images2 = crop_image_matrix_val[:,index]
# with tf.device('/device:GPU:0'):
#     temp_pred2_val2 = cascade_model2.predict(temp_val_images2) # (200,2,1)
# temp_pred2_val2_store = temp_pred2_val2*actual_reshape_size_matrix_val/227
# val_pred_matrix[:,:,index] = temp_pred2_val2_store[:,:,index]

In [ ]:
# forth joint
# index = 3
# temp_images3 = crop_image_matrix[:,index]
# temp_true_labels3 = scaled_trues_label[:,index,:]
# temp_images_val3 = crop_image_matrix_val[:,index]
# temp_true_labels3_val = scaled_trues_label_val[:,index,:]
# cascade_model3 = build_cascade_model()
# cascade_model3.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss, metrics=[])

In [ ]:
# csv_logger = CSVLogger('stage2_3_v2.csv', append=True, separator=';')

In [ ]:
# with tf.device('/device:GPU:0'):
    # fit the model
#     cascade_model3.fit(temp_images3, temp_true_labels3, epochs=50,batch_size=64,validation_data = (temp_images_val3,temp_true_labels3_val),callbacks=[csv_logger])

In [ ]:
# cascade_model3.save('stage2_3_v2.h5')

In [ ]:
# store training prediction
# with tf.device('/device:GPU:0'):
#     temp_pred2_train3 = cascade_model3.predict(temp_images3) # (1600,2,1)
# # scale back to original images to compute loss
# temp_pred2_train3_store = temp_pred2_train3*actual_reshape_size_matrix/227
# train_pred_matrix[:,:,index] = temp_pred2_train3_store[:,:,index]
# temp_val_images3 = crop_image_matrix_val[:,index]
# with tf.device('/device:GPU:0'):
#     temp_pred2_val3 = cascade_model3.predict(temp_val_images3) # (200,2,1)
# temp_pred2_val3_store = temp_pred2_val3*actual_reshape_size_matrix_val/227
# val_pred_matrix[:,:,index] = temp_pred2_val3_store[:,:,index]

In [ ]:
# fifth joint
# index = 4
# temp_images4 = crop_image_matrix[:,index]
# temp_true_labels4 = scaled_trues_label[:,index,:]
# temp_images_val4 = crop_image_matrix_val[:,index]
# temp_true_labels4_val = scaled_trues_label_val[:,index,:]
# cascade_model4 = build_cascade_model()
# cascade_model4.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss, metrics=[])

In [ ]:
# csv_logger = CSVLogger('stage2_4_v2.csv', append=True, separator=';')

In [ ]:
# with tf.device('/device:GPU:0'):
    # fit the model
#     cascade_model4.fit(temp_images4, temp_true_labels4, epochs=50,batch_size=64,validation_data = (temp_images_val4,temp_true_labels4_val),callbacks=[csv_logger])

In [ ]:
# cascade_model4.save('stage2_4_v2.h5')

In [ ]:
# store training prediction
# with tf.device('/device:GPU:0'):
#     temp_pred2_train4 = cascade_model4.predict(temp_images4) # (1600,2,14)
# # scale back to original images to compute loss
# temp_pred2_train4_store = temp_pred2_train4*actual_reshape_size_matrix/227
# train_pred_matrix[:,:,index] = temp_pred2_train4_store[:,:,index]
# temp_val_images4 = crop_image_matrix_val[:,index]
# with tf.device('/device:GPU:0'):
#     temp_pred2_val4 = cascade_model4.predict(temp_val_images4) # (200,2,14)
# temp_pred2_val4_store = temp_pred2_val4*actual_reshape_size_matrix_val/227
# val_pred_matrix[:,:,index] = temp_pred2_val4_store[:,:,index]

In [ ]:
# sixth joint
# index = 5
# temp_images5 = crop_image_matrix[:,index]
# temp_true_labels5 = scaled_trues_label[:,index,:]
# temp_images_val5 = crop_image_matrix_val[:,index]
# temp_true_labels5_val = scaled_trues_label_val[:,index,:]
# cascade_model5 = build_cascade_model()
# cascade_model5.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss, metrics=[])

In [ ]:
# csv_logger = CSVLogger('stage2_5_v2.csv', append=True, separator=';')

In [ ]:
# with tf.device('/device:GPU:0'):
    # fit the model
#     cascade_model5.fit(temp_images5, temp_true_labels5, epochs=50,batch_size=64,validation_data = (temp_images_val5,temp_true_labels5_val),callbacks=[csv_logger])

In [ ]:
# cascade_model5.save('stage2_5_v2.h5')

In [ ]:
# store training prediction
# with tf.device('/device:GPU:0'):
#     temp_pred2_train5 = cascade_model5.predict(temp_images5) # (1600,2,14)
# # scale back to original images to compute loss
# temp_pred2_train5_store = temp_pred2_train5*actual_reshape_size_matrix/227
# train_pred_matrix[:,:,index] = temp_pred2_train5_store[:,:,index]
# temp_val_images5 = crop_image_matrix_val[:,index]
# with tf.device('/device:GPU:0'):
#     temp_pred2_val5 = cascade_model5.predict(temp_val_images5) # (200,2,14)
# temp_pred2_val5_store = temp_pred2_val5*actual_reshape_size_matrix_val/227
# val_pred_matrix[:,:,index] = temp_pred2_val5_store[:,:,index]

In [ ]:
# seventh joint
# index = 6
# temp_images6 = crop_image_matrix[:,index]
# temp_true_labels6 = scaled_trues_label[:,index,:]
# temp_images_val6 = crop_image_matrix_val[:,index]
# temp_true_labels6_val = scaled_trues_label_val[:,index,:]
# cascade_model6 = build_cascade_model()
# cascade_model6.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss, metrics=[])
# csv_logger = CSVLogger('stage2_6_v2.csv', append=True, separator=';')

In [ ]:
# with tf.device('/device:GPU:0'):
#     # fit the model
#     cascade_model6.fit(temp_images6, temp_true_labels6, epochs=50,batch_size=64,validation_data = (temp_images_val6,temp_true_labels6_val),callbacks=[csv_logger])

In [ ]:
# cascade_model6.save('stage2_6_v2.h5')

In [ ]:
# store training prediction
# with tf.device('/device:GPU:0'):
#     temp_pred2_train6 = cascade_model6.predict(temp_images6) # (1600,2,14)
# # scale back to original images to compute loss
# temp_pred2_train6_store = temp_pred2_train6*actual_reshape_size_matrix/227
# train_pred_matrix[:,:,index] = temp_pred2_train6_store[:,:,index]
# temp_val_images6 = crop_image_matrix_val[:,index]
# with tf.device('/device:GPU:0'):
#     temp_pred2_val6 = cascade_model6.predict(temp_val_images6) # (200,2,14)
# temp_pred2_val6_store = temp_pred2_val6*actual_reshape_size_matrix_val/227
# val_pred_matrix[:,:,index] = temp_pred2_val6_store[:,:,index]

In [ ]:
# eighth joint
# index = 7
# temp_images7 = crop_image_matrix[:,index]
# temp_true_labels7 = scaled_trues_label[:,index,:,index,None]
# cascade_model7 = build_cascade_model()
# cascade_model7.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss, metrics=[])
# csv_logger = CSVLogger('stage2_7.csv', append=True, separator=';')

In [ ]:
# with tf.device('/device:GPU:0'):
#     # fit the model
#     cascade_model7.fit(temp_images7, temp_true_labels7, epochs=50,batch_size=64,callbacks=[csv_logger])

In [ ]:
# cascade_model7.save('stage2_6.h5')

In [ ]:
# store training prediction
# with tf.device('/device:GPU:0'):
#     temp_pred2_train7 = cascade_model7.predict(temp_images7) # (1600,2,1)
# # scale back to original images to compute loss
# train_pred_matrix[:,:,index,None] = temp_pred2_train7*actual_reshape_size_matrix[:,:,index,None]/227

# temp_val_images7 = crop_image_matrix_val[:,index]
# with tf.device('/device:GPU:0'):
#     temp_pred2_val7 = cascade_model7.predict(temp_val_images7) # (200,2,1)
# val_pred_matrix[:,:,index,None] = temp_pred2_val7*actual_reshape_size_matrix_val[:,:,index,None]/227

In [ ]:
# ninth joint
# index = 8
# temp_images7 = crop_image_matrix[:,index]
# temp_true_labels8 = scaled_trues_label[:,index,:,index,None]
# cascade_model8 = build_cascade_model()
# cascade_model8.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss, metrics=[])
# csv_logger = CSVLogger('stage2_8.csv', append=True, separator=';')

In [ ]:
# with tf.device('/device:GPU:0'):
#     # fit the model
#     cascade_model8.fit(temp_images8, temp_true_labels8, epochs=50,batch_size=64,callbacks=[csv_logger])

In [ ]:
# cascade_model7.save('stage2_7.h5')

In [ ]:
# store training prediction
# with tf.device('/device:GPU:0'):
#     temp_pred2_train8 = cascade_model8.predict(temp_images8) # (1600,2,1)
# # scale back to original images to compute loss
# train_pred_matrix[:,:,index,None] = temp_pred2_train8*actual_reshape_size_matrix[:,:,index,None]/227

# temp_val_images8 = crop_image_matrix_val[:,index]
# with tf.device('/device:GPU:0'):
#     temp_pred2_val8 = cascade_model8.predict(temp_val_images8) # (200,2,1)
# val_pred_matrix[:,:,index,None] = temp_pred2_val8*actual_reshape_size_matrix_val[:,:,index,None]/227

In [ ]:
##################################################################

In [ ]:
# the following code put every model together
# epoches = 10
# train_pred_matrix = np.zeros((1600,2,14))
# val_pred_matrix = np.zeros((200,2,14))
# for epoch in range(epoches):
    
#     for i in range(14):
        
#         # prepare the data
#         # crop_image_matrix (1600,14,227,227,3) => (1600,227,227,3) extract all images belong to one specific bounding box
#         temp_images = crop_image_matrix[:,i]
#         # scaled trues label (1600,14,2,14) => (1600,i,2,i) => (1600,2,1)
#         temp_true_labels = scaled_trues_label[:,i,:,i,None]
        
#         # create the model
#         cascade_model = build_cascade_model()
        
#         # compile the model
#         cascade_model.compile(optimizer=Adam(lr=0.0005), loss=euclidean_distance_loss, metrics=[])
        
#         with tf.device('/device:GPU:0'):
#             # fit the model
#             history = cascade_model.fit(temp_images, temp_true_labels, epochs=1,batch_size=64)
        
#         # store training prediction
#         with tf.device('/device:GPU:0'):
#             temp_pred2_train = cascade_model.predict(temp_images) # (1600,2,1)
        
#         # scale back to original images to compute loss
#         temp_train_pred2 = temp_pred2_train*actual_reshape_size_matrix[:,:,i,None]/227
        
#         train_pred_matrix[:,:,i,None] = temp_train_pred2
        
#         # store validation prediction
        
#         temp_val_images = crop_image_matrix_val[:,i]
#         with tf.device('/device:GPU:0'):
#             temp_pred2_val = cascade_model.predict(temp_val_images) # (200,2,1)
            
#         temp_val_pred2 = temp_pred2_val*actual_reshape_size_matrix_val[:,:,i,None]/227
        
#         val_pred_matrix[:,:,i,None] = temp_val_pred2
        
        
#     temp_train_loss = K.sqrt(K.sum(K.square(train_pred_matrix - train_label)))/1600
        
#     print( 'training loss is ', temp_train_loss)
    
#     temp_val_loss = K.sqrt(K.sum(K.square(val_pred_matrix - val_label)))/200
            